In [67]:
import os
import glob
import numpy as np
import pandas as pd

#### GTEx data

In [68]:
# tr = transcript ratio;  ec = exon correlation;
formatted_tr = "/home/lzg/1906/QTL_data/unfiltered/sQTL_transcript.txt"
raw_dir = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/GTEx"
trans_ratio = "%s/sQTLSeekeR_Transcript-Ratio/sQTLSeekeR_Transcript-Ratio.csv" % raw_dir

In [69]:
# process trans_ratio
def generate_tr_dict():
    df = pd.read_csv(trans_ratio)
    df["query_id"] = df["Gene Symbol"].astype(str) + ";" + df["SNP"].astype(str)
    query_list = [x.strip() for x in df["query_id"].tolist()] # remove the end blank of SNP
    df["value"] = "Transcript-Ratio" + ";" + df["Transcript 1"].str.split(".").str[0] + ";" + df["Transcript 2"].str.split(".").str[0]
    tr_dict = dict(zip(query_list, df["value"]))
    print(tr_dict["C1orf201;rs1016209"]) # supposed to print out Transcript-Ratio;ENST00000337248.4;ENST00000003583.8
    return tr_dict

In [70]:
def generate_ensembl_coordinate():
    gff = "/data/database/GRCh38/GENCODE/gencode.v27.annotation.gff3"
    df = pd.read_table(gff, sep="\t", comment="#", header=None)
    df_tc = df[df.iloc[:, 2] == "transcript"]
    df_tc["Ensembl_ID"] = df_tc.iloc[:, 8].str.split("ID=").str[1].str.split(".").str[0]
    df_tc["coor"] = df_tc.iloc[:, 0] + ":" + df_tc.iloc[:, 3].astype(str) + "-" + df_tc.iloc[:, 4].astype(str)
    ens_coor_dict = dict(zip(df_tc["Ensembl_ID"], df_tc["coor"]))
    print("\t" + ens_coor_dict["ENST00000456328"]) # supposed to be chr1:11869-14409
    return ens_coor_dict

In [71]:
def add_id_for_TranscriptRatio():
    dict_tr = generate_tr_dict()
    ens_coor_dict = generate_ensembl_coordinate()
    #
    df_tr = pd.read_table(formatted_tr, sep="\t", skiprows=1)
    df_tr["query_id"] = df_tr["Gene_Name"].astype(str) + ";" + df_tr["RS_ID"].astype(str)
    query_list =[x.strip() for x in df_tr["query_id"].tolist()] # remove the end blank space;
    id_list = [dict_tr[x] if x in dict_tr else str(np.nan) for x in query_list]
    coor_list = []
    for sp_id in id_list:
        pre = sp_id.split(";")[0]
        post = ";".join([ens_coor_dict[x] if x in ens_coor_dict else str(np.nan) for x in sp_id.split(";")[1:]])
        coor_list.append(pre + ";" + post)
    df_tr["Splicing_ID"] = coor_list
    # can't decide sQTL type
    df_tr["sQTL_type"] = "cis"
    df_tr["Cutoff"] = "5kb upstream or downstream of each gene's transcript start or end sites"
    del df_tr['query_id']
    df_tr.to_csv("/home/galaxy/project/m6AQTL/data/sqtl/total_data/sQTL_TranscriptRatio.txt", sep="\t", index=False)
    print(df_tr.head())

In [72]:
add_id_for_TranscriptRatio()

Transcript-Ratio;ENST00000337248;ENST00000003583


/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


	chr1:11869-14409
        Ensembl_ID Gene_Name  Position Splicing_ID  \
0  ENSG00000001460     STPG1       NaN        nan;   
1  ENSG00000001460     STPG1       NaN        nan;   
2  ENSG00000001460     STPG1       NaN        nan;   
3  ENSG00000001460     STPG1       NaN        nan;   
4  ENSG00000001460     STPG1       NaN        nan;   

                                        Event_type        RS_ID  REF  ALT  \
0                                    exon skipping   rs1016209   NaN  NaN   
1                                    exon skipping  rs11589294   NaN  NaN   
2                                    exon skipping  rs11587667   NaN  NaN   
3                                    exon skipping   rs7529526   NaN  NaN   
4  complex event 5';complex event;complex event 3'  rs12140084   NaN  NaN   

  sQTL_type                                             Cutoff    Pvalue  \
0       cis  5kb upstream or downstream of each gene's tran...  0.000024   
1       cis  5kb upstream or downstream of

In [129]:
# process ExonCorrelation
ec = "/home/lzg/1906/QTL_data/unfiltered/sQTL_exon.txt"
df = pd.read_table(ec, sep="\t", skiprows=1)
df.head()

,Ensembl_ID,Gene_Name,Position,Splicing_ID,Event_type,RS_ID,REF,ALT,sQTL_type,Cutoff,Pvalue,Corrected_pvalue,Measure,Value,Cell_or_tissue,Condition,Experimental_tag,PubMed_ID
0,ENSG00000116906,GNPAT,NaN,chr1:231403426-231403649;chr1:231409668-231409808,NaN,rs10779813,NaN,NaN,cis,1M,9.810000e-07,NaN,Spearman Rho,0.449,Artery Tibial,NaN,2,25954001
1,ENSG00000000419,DPM1,NaN,chr20:49574900-49575058;chr20:49562274-49562299,NaN,rs73130786,NaN,NaN,cis,1M,2.090000e-06,NaN,Spearman Rho,-0.446,Thyroid,NaN,8,25954001
2,ENSG00000000457,SCYL3,NaN,chr1:169822215-169822913;chr1:169823411-169824105,NaN,rs11585877,NaN,NaN,cis,1M,9.010000e-07,NaN,Spearman Rho,-0.450,Artery Tibial,NaN,2,25954001
3,ENSG00000000457,SCYL3,NaN,chr1:169831754-169831938;chr1:169822215-169822913,NaN,rs10753798,NaN,NaN,cis,1M,1.170000e-06,NaN,Spearman Rho,-0.473,Skin Sun Exposed (Lower leg),NaN,7,25954001
4,ENSG00000000971,CFH,NaN,chr1:196658550-196658744;chr1:196654194-196654367,NaN,rs10921983,NaN,NaN,cis,1M,8.480000e-07,NaN,Spearman Rho,0.482,Adipose Subcutaneous,NaN,10,25954001


In [130]:
raw_beds = df["Splicing_ID"].tolist()
raw_list = []
for i in range(len(raw_beds)):
    a, b = raw_beds[i].split(";")[0], raw_beds[i].split(";")[1]
    a_1 = "\t".join([a.split(":")[0], str(a.split(":")[1].split("-")[0]), str(a.split("-")[1]), str(i)+"."+str(1)])
    b_2 = "\t".join([b.split(":")[0], str(b.split(":")[1].split("-")[0]), str(b.split("-")[1]), str(i)+"."+str(2)])
    raw_list.append(a_1)
    raw_list.append(b_2)
grch37_bed = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/liftover/grch37.bed"
with open(grch37_bed, 'w') as fw:
    fw.writelines(["%s\n" % x for x in raw_list])

In [131]:
# ### liftover
chain_file, failure_file = "/data/database/chain_file/hg19ToHg38.over.chain", "/home/galaxy/failure.txt"
grch37_bed = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/liftover/grch37.bed"
grch38_bed = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/liftover/grch38.bed"
command = "liftOver %s %s %s %s" % (grch37_bed, chain_file, grch38_bed, failure_file)
os.system(command)

0

In [132]:
line_num = len(df)
print(line_num)
df_38 = pd.read_table(grch38_bed, sep="\t", header=None)
df_38.columns = ["chr_name", "start", "end", "line_num"]
print(df_38.head())
full_list = []
for i in range(line_num):
    full_list.append([i+0.1])
    full_list.append([i+0.2])
df_full = pd.DataFrame(full_list)
df_full.columns = ["line_num"]
df_outer = df_38.merge(df_full, on="line_num", how="right")
df_outer = df_outer.sort_values(["line_num"])
df_outer["id"] = df_outer["chr_name"] + ":" + df_outer["start"].astype(str).str.split(".").str[0] + "-" + df_outer["end"].astype(str).str.split(".").str[0]
print(df_outer.tail())
id_list = df_outer["id"].tolist()
result_list = []
for i in range(0, len(id_list)-1, 2):
    result_list.append(";".join(["ExonCorrelation", str(id_list[i]), str(id_list[i+1])]))
df_1 = df
assert len(df_1["Splicing_ID"]) == len(result_list)
df_1["Splicing_ID"] = result_list
df_1.to_csv("/home/galaxy/project/m6AQTL/data/sqtl/total_data/sQTL_ExonCorrelation.txt", sep="\t", index=False)

41185
  chr_name      start        end  line_num
0     chr1  231267680  231267903       0.1
1     chr1  231273922  231274062       0.2
2    chr20   50958363   50958521       1.1
3    chr20   50945737   50945762       1.2
4     chr1  169853074  169853772       2.1
      chr_name       start         end  line_num                      id
82286    chr16   4407562.0   4407684.0   41182.2   chr16:4407562-4407684
82287    chr19   7470126.0   7470245.0   41183.1   chr19:7470126-7470245
82288    chr19   7476875.0   7477436.0   41183.2   chr19:7476875-7477436
82289     chr6  31669118.0  31669172.0   41184.1  chr6:31669118-31669172
82290     chr6  31665861.0  31666283.0   41184.2  chr6:31665861-31666283


#### process paper

In [133]:
raw_file = "/home/lzg/1906/QTL_data/unfiltered/sQTL_paper.txt"
df = pd.read_table(raw_file, sep="\t", skiprows=1)
print(len(df))
df.head()

20417


,Ensembl_ID,Gene_Name,Position,Splicing_ID,Event_type,RS_ID,REF,ALT,sQTL_type,Cutoff,Pvalue,Corrected_pvalue,Measure,Value,Cell_or_tissue,Condition,Experimental_tag,PubMed_ID
0,ENSG00000143337,TOR1AIP1,NaN,chr1:179884769-179889310:chr1:179884769-179889313,A3,rs2245425,NaN,NaN,NaN,NaN,1.611478e-69,NaN,NaN,NaN,peripheral blood mononuclear cells(PBMCs),NaN,1.0,30975994.0
1,ENSG00000181381,DDX60L,NaN,chr4:168396127-168400826:chr4:168396124-168400826,A3,rs3857035,NaN,NaN,NaN,NaN,2.642871e-69,NaN,NaN,NaN,peripheral blood mononuclear cells(PBMCs),NaN,1.0,30975994.0
2,ENSG00000181381,DDX60L,NaN,chr4:168375524-168377728:chr4:168377843-168378354,SE,rs2046770,NaN,NaN,NaN,NaN,2.451629e-31,NaN,NaN,NaN,peripheral blood mononuclear cells(PBMCs),NaN,1.0,30975994.0
3,ENSG00000181381,DDX60L,NaN,chr4:168472809-168475996:chr4:168476120-168480377,SE,rs4692942,NaN,NaN,NaN,NaN,1.732243e-18,NaN,NaN,NaN,peripheral blood mononuclear cells(PBMCs),NaN,1.0,30975994.0
4,ENSG00000181381,DDX60L,NaN,chr4:168461930-168461931:chr4:168462040-168471...,AL,rs56999040,NaN,NaN,NaN,NaN,3.147239e-08,NaN,NaN,NaN,peripheral blood mononuclear cells(PBMCs),NaN,1.0,30975994.0


In [134]:
#########################################################################################################
chain_file, failure_file = "/data/database/chain_file/hg19ToHg38.over.chain", "/home/galaxy/failure.txt"
raw_bed = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/liftover/grch37.bed"
final_bed = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/liftover/grch38.bed"
id_list = df["Splicing_ID"].tolist()
result_file = "/home/galaxy/project/m6AQTL/data/sqtl/total_data/sQTL_paper.txt"
# replace hg19 coordinates with hg38;
#########################################################################################################


def map_to_original():
    df_38 = pd.read_table(final_bed, sep="\t", header=None)
    df_38.columns = ["chr_name", "start", "end", "line_num"]
    df_full = pd.DataFrame(
        pd.read_table(raw_bed, sep="\t", header=None).iloc[:, -1])
    df_full.columns = ["line_num"]
    df_outer = df_38.merge(df_full, on="line_num", how="right")
    df_outer["id"] = df_outer["chr_name"] + ":" + df_outer["start"].astype(
        str).str.split(".").str[0] + "-" + df_outer["end"].astype(
            str).str.split(".").str[0]
    print(df_outer.head())
    id_list = df_outer["id"].tolist()
    line_nums = df_outer["line_num"].tolist()
    result_list = []
    for i in range(1, len(df)+1):
        result_list.append(";".join([
            str(id_list[j]) for j in range(len(id_list))
            if str(line_nums[j]).startswith(str(i) + ".")
        ]))
    df_1 = df
    assert len(df_1["Splicing_ID"]) == len(result_list)
    df_1["Splicing_ID"] = result_list
    #
    df_1["Experimental_tag"] = df_1["Experimental_tag"].astype(int)
    df_1["PubMed_ID"] = df_1["PubMed_ID"].astype(int)
    df_1.to_csv(result_file, sep="\t", index=False)


def liftover():
    command = "liftOver %s %s %s %s" % (raw_bed, chain_file, final_bed, failure_file)
    os.system(command)


def add_peakNumber(query_list, result_file):
    result_list = []
    for i in range(len(query_list)):
        query = query_list[i]
        if str(query).startswith("chr"):
            raw_list = add_peak_number(query, i + 1)
            result_list += raw_list
    print(len(result_list))
    with open(result_file, 'w') as fw:
        fw.writelines(["%s\n" % x for x in result_list])


def add_peak_number(line, line_num):
    #     a = "chr10:3103945-3105114"
    #     b = "chr9:137428616-137428882"
    #     c = "chr12:70344228-70346180:chr12:70346625-70346626:chr12:70344228-70353829:chr12:70353912-70353913"
    peak_list = ["chr%s" % x.strip(":") for x in line.split("chr")[1:]]
    result_list, decimal = [], 0.1
    for i in range(len(peak_list)):
        peak_num = str(line_num + decimal * (i + 1))
        # separator - or +
        bed = "\t".join([
            peak_list[i].split(":")[0],
            str(peak_list[i].split(":")[1].split("-")[0]),
            str(peak_list[i].replace("+", "-").split("-")[1].split(":")[0]),
            peak_num
        ])
        result_list.append(bed)
    return result_list


add_peakNumber(id_list, raw_bed)
liftover()
map_to_original()

19700
  chr_name        start          end  line_num                        id
0     chr1  179915634.0  179920175.0       1.1  chr1:179915634-179920175
1     chr1  179915634.0  179920178.0       1.2  chr1:179915634-179920178
2     chr4  167474976.0  167479675.0       2.1  chr4:167474976-167479675
3     chr4  167474973.0  167479675.0       2.2  chr4:167474973-167479675
4     chr4  167454373.0  167456577.0       3.1  chr4:167454373-167456577
